In [100]:
import pandas as pd

In [101]:
# load dataframe mvp1
mvp1_path = "web-dash\mvp1data.csv"
df = pd.read_csv(mvp1_path)

In [102]:
#group df in each topic so it easy to select first sample(first 30 sentences)
grouped_df = df.groupby('topic')

In [103]:
# Create a new column "selected" and initialize it with 0
df['selected'] = 0

# Iterate through each group (topic) and mark up to 30 rows as selected
for topic, group in grouped_df:
    selected_indices = group.head(100).index
    df.loc[selected_indices, 'selected'] = 1

In [104]:
#select the selected row
selected_rows = df[df['selected'] == 1]

# Save the selected rows to a new CSV file
selected_rows.to_csv('web-dash\mvp2data.csv', index=False)

Convert into input.pl

In [105]:
marked_dataset = "web-dash\mvp2data.csv"
df_marked = pd.read_csv(marked_dataset)

In [106]:
df_marked.value_counts("topic")

topic
COVID                         100
Integrity                     100
Supreme Court                 100
Racism                        100
National Security             100
Minimum wage                  100
Climate change                100
Why They Should Be Elected    100
Healthcare                    100
Families                      100
Election                      100
Economy                       100
Taxes                          81
Vice-presidency                62
Democracy                      50
LGBTQ                          32
Foreign policy                 17
Leadership                     10
dtype: int64

In [107]:
#list a topics name
group_marked = df_marked.groupby('topic')
list_topic = []
for topic, group in group_marked:
    list_topic.append(topic)

In [108]:
#create a unique dictionary betweeb sentence 1 and sentence 2
combined_list = [x for n in (df_marked["sentence_1"],df_marked["sentence_2"]) for x in n]
unique_list = list(set(combined_list))
element_to_integer = {element: str(index + 1) for index, element in enumerate(unique_list)}
g_description = {value: key for key, value in element_to_integer.items()}

In [109]:
# set sen1_inx and sen2_inx into -1 then after i assign all index we can check is there any unassign index or not
df_marked['sen1_inx'] = -1
df_marked['sen2_inx'] = -1
for key, value in g_description.items():
    for i in range(len(df_marked["sentence_1"])):
        if value == df_marked["sentence_1"][i]:
            df_marked['sen1_inx'][i] = key
        if value == df_marked["sentence_2"][i]:
            df_marked['sen2_inx'][i] = key

/var/folders/s0/79hxvcq50xzchv821cc55yxw0000gp/T/ipykernel_96553/2767702563.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_marked['sen1_inx'][i] = key
/var/folders/s0/79hxvcq50xzchv821cc55yxw0000gp/T/ipykernel_96553/2767702563.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_marked['sen2_inx'][i] = key


In [110]:
# check is there any unassign index in dataframe both sen1 and sen2
if any(df_marked["sen1_inx"] == -1):
    print("There is at least one occurrence of -1 in df_marked['sen1_inx'].")
else:
    print("There is no occurrence of -1 in df_marked['sen1_inx'].")

if any(df_marked["sen2_inx"] == -1):
    print("There is at least one occurrence of -1 in df_marked['sen2_inx'].")
else:
    print("There is no occurrence of -1 in df_marked['sen2_inx'].")

There is no occurrence of -1 in df_marked['sen1_inx'].
There is no occurrence of -1 in df_marked['sen2_inx'].


In [111]:
def find_topic(topic, model):
    #map sentences and relation
    senten = []
    rela = []
    for i in range(len(df_marked["topic"])):
        if df_marked["topic"][i] == topic:
            ind_1 = list(g_description.keys())[list(g_description.values()).index(df_marked["sentence_1"][i])]
            ind_2 = list(g_description.keys())[list(g_description.values()).index(df_marked["sentence_2"][i])]
            if ind_1 not in senten: #check is there any sen1 in senten or not to prevent duplicated
                senten.append(ind_1)
            if ind_2 not in senten:
                senten.append(ind_2)
            if df_marked[model][i]:
                rela.append(f"{ind_1},{ind_2}")
    return senten, rela

def convert_into_input(sen, rel, filename):
    #convert senten and rela into input file format
    ipl =""
    for i in sen:
        ipl = ipl+f"arg({i})."
    for i in rel:
        ipl = ipl+f"att({i})."
    with open(filename, 'w') as file:
        file.write(ipl)

In [112]:
def loop_topic(model):
    for topic in list_topic:
        sentence, relation = find_topic(topic, model)
        filename = f"{model}\input_{topic}.pl"
        filename = filename.replace(" ", "_")
        convert_into_input(sentence, relation, filename)

In [113]:
loop_topic("gpt_davinci")

In [114]:
loop_topic("gpt_35")

In [115]:
loop_topic("gpt_35_0613")

In [116]:
loop_topic("relation_AAF")

In [117]:
df_marked.to_csv('web-dash\mvp2data.csv', index=False)

### Mark the result into CSV file


In [119]:
marked_dataset = "web-dash\mvp2data.csv"
df_marked = pd.read_csv(marked_dataset)

In [120]:
df_marked

,_unit_id,_golden,_unit_state,_trusted_judgments,confidence,confidence:stddev,relation,relation:confidence,orig__golden,context,...,speaker_2,topic,relation_AAF,relation_BAF,gpt_davinci,gpt_35,gpt_35_0613,selected,sen1_inx,sen2_inx
0,3085729585,True,golden,65,4.45,0.702,neither,0.8807,True,"<strong>Kamala Harris:</strong> ""Please vote.""...",...,Mike Pence,Election,0,0,2,2,2,1,738,1547
1,3085729586,True,golden,70,4.46,0.750,neither,0.9203,True,"<strong>Donald Trump:</strong> ""Look what happ...",...,Joe Biden,Racism,0,0,2,2,2,1,1384,352
2,3085729587,True,golden,60,3.93,0.772,neither,1.0000,True,"<strong>Donald Trump:</strong> ""And you’ll get...",...,Joe Biden,Economy,0,0,0,0,0,1,143,1631
3,3085729588,False,golden,6,4.33,0.471,attack,0.4744,True,"<strong>Kristen Welker:</strong> ""I have one f...",...,Donald Trump,Climate change,1,1,0,0,0,1,1062,1801
4,3085729589,True,golden,67,4.57,0.628,support,0.9631,True,"<strong>Susan Page:</strong> ""So, tell us spec...",...,Mike Pence,Supreme Court,0,2,2,2,2,1,353,230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1447,3101684951,False,finalized,6,4.17,0.687,neither,0.8339,NaN,"<strong>Donald Trump:</strong> ""He was thrown ...",...,Joe Biden,Why They Should Be Elected,0,0,0,0,0,1,86,1372
1448,3101684954,False,finalized,4,4.00,0.707,neither,0.7530,NaN,"<strong>Joe Biden:</strong> ""And what’s happen...",...,Joe Biden,LGBTQ,0,0,0,0,0,1,475,371
1449,3101684963,False,finalized,3,3.00,0.000,neither,1.0000,NaN,"<strong>Donald Trump:</strong> ""I don’t love w...",...,Savannah Guthrie,Taxes,0,0,0,0,0,1,1003,569
1450,3101685003,False,finalized,3,3.00,0.000,neither,1.0000,NaN,"<strong>Donald Trump:</strong> ""I am under aud...",...,Savannah Guthrie,Taxes,0,0,0,0,0,1,941,312


In [121]:
df_marked.value_counts("topic")

topic
COVID                         100
Integrity                     100
Supreme Court                 100
Racism                        100
National Security             100
Minimum wage                  100
Climate change                100
Why They Should Be Elected    100
Healthcare                    100
Families                      100
Election                      100
Economy                       100
Taxes                          81
Vice-presidency                62
Democracy                      50
LGBTQ                          32
Foreign policy                 17
Leadership                     10
dtype: int64

In [80]:
test = [1, 2, "3"]
print(1 in test)
print(3 in test)

True
False


In [122]:
def markcsv(number, model):

    # df_marked[f'satis_{model}'] = 0
    for i in range(len(df_marked["sen1_inx"])):
        if df_marked["sen1_inx"][i] in number:
            df_marked[f'satis_{model}'][i] = 1

def readline(file_name, model):
    file1 = open(file_name, 'r')
    Lines = file1.readlines()
    count = 0
    numbers = []
    # Strips the newline character
    for line in Lines:
        count += 1
        word = line.split(" ")
        for i in word:
            if "in(" in i:
                number = i[3:len(i)-1]
                # print(number)
                numbers.append(int(number))
    markcsv(numbers, model)
    return numbers

        # print("Line{}: {}".format(count, line.strip()))

In [123]:
import os
path = "ASP_output"
folder_path = os.listdir(path)
for i in folder_path:
    print(i)
    print("=============")
    model_path = path+"\\"+i
    txt_path = os.listdir(model_path)
    for l in txt_path:
        filename = model_path+"\\"+l
        numbers = readline(filename, i)
        print(numbers)
    # break

relation_AAF
[1318, 1006, 1702, 1603, 1809, 281, 1552, 73, 330, 757, 131, 930, 932, 944, 981, 232, 1322, 1404, 1567, 1475, 1437, 1024, 1752]
[1762, 1285, 69, 1732, 1573, 1802, 241, 570, 457, 229, 604, 1733, 411, 7, 1082, 262, 1349, 1589, 1265, 408, 794, 1441, 65, 1299, 839, 1027, 1842, 315, 1256, 873, 1248, 48, 495, 1555, 563, 1537, 1140, 1494, 14, 1143, 1657, 1210, 399, 1587, 88, 849, 703, 244, 480, 475, 29, 807, 1613, 431, 1739, 1174, 908, 172, 209, 267, 1452, 1015, 285, 1411, 660, 13, 1708, 242, 1523, 1295, 887, 1535, 917, 958, 317, 792, 1406, 1619, 1249, 1737, 1730, 1485, 591, 612, 736, 1344, 711, 447, 1834, 942, 838, 147, 1807, 249, 252, 1346, 532, 51, 617, 671, 868, 862, 108, 210, 1514, 1032, 682, 295, 1197, 270, 835, 111, 465, 1472, 576, 155, 669, 1412, 354, 97, 279, 1268, 622, 611, 297, 979, 114, 128, 550, 1502, 556, 1451, 141, 432, 1568, 543, 592, 1768, 1620, 940, 1646, 1251, 896, 424, 348, 217, 280, 80, 637, 1148, 1767, 1711, 503, 1094, 167, 1677, 1720, 949, 1825, 590, 159, 1

/var/folders/s0/79hxvcq50xzchv821cc55yxw0000gp/T/ipykernel_96553/657226236.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_marked[f'satis_{model}'][i] = 1


[1381, 1020, 659, 194, 86, 850, 1165, 1714, 783, 1226, 1407, 1840, 1111, 1748, 633, 748, 680, 1372, 413, 1769, 176, 1676, 1477, 248, 1108, 1371, 1258, 1216, 71, 1489, 107, 1237, 138, 876, 114, 1511, 47, 992, 34, 1220, 374, 243, 1505, 1647, 1721, 350, 362, 453, 975, 38, 90, 989, 1146, 1572, 549, 504, 1790, 103, 1705, 382, 1413, 1218, 1585, 152, 560, 1173, 443, 563, 1479, 1723, 110, 1066, 1035, 464, 1725, 1011, 1664, 1217, 1223, 689, 1637]
[1580, 471, 368, 1255, 1304, 988, 1593, 1037, 178, 1459, 238, 565, 1583, 1221, 788, 1597, 1706, 619, 1231, 1774, 1712, 953, 1828, 1240, 1056, 1331, 1267, 609, 1473, 1212, 77, 1756, 1126, 1264, 1144, 239, 1429, 189, 643, 684, 421, 947, 599, 266, 1277, 847, 1340, 678, 1759, 1728, 82, 931, 968, 630, 287, 1172, 35, 779, 744, 638, 1415, 1271, 420, 183, 1594, 1368, 878, 460, 366, 149, 768, 282, 1527, 314, 1655, 538, 156, 1257, 440, 201, 1566, 1821, 627, 1402, 221, 1461, 777, 129, 1380, 690, 561, 1519, 346, 1598, 1030, 1336, 1042, 1288, 1491, 1695, 289, 139, 

/var/folders/s0/79hxvcq50xzchv821cc55yxw0000gp/T/ipykernel_96553/657226236.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_marked[f'satis_{model}'][i] = 1


[838, 1645, 1649, 105, 1387, 1570, 1504, 1482, 1395, 534, 1113, 1600, 21, 1682, 590, 1825, 949, 1720, 167, 1094, 503, 1711, 1148, 637, 80, 280, 217, 348, 896, 1646, 940, 1620, 1768, 592, 543, 1568, 141, 1451, 556, 1502, 550, 128, 114, 979, 297, 622, 279, 354, 669, 155, 576, 1472, 465, 111, 835, 270, 1197, 295, 682, 1032, 1514, 108, 862, 868, 671, 617, 51, 532, 1346, 252, 249, 1807, 147, 942, 1834, 447, 711, 736, 612, 591, 1485, 1730, 1737, 1249, 1619, 1406, 792, 317, 958, 1535, 887, 1295, 1523, 242, 13, 660, 1411, 285, 1015, 1452, 267, 209, 172, 908, 1174, 1739, 1613, 807, 29, 475, 480, 244, 703, 88, 1587, 1210, 1143, 14, 1494, 1140, 1537, 1555, 495, 1248, 873, 315, 1842, 839, 1299, 65, 1441, 408, 1265, 1589, 262, 1082, 7, 411, 1733, 604, 229, 457, 570, 241, 1802, 1573, 1732, 1285, 399]
[109, 1062, 1801, 290, 605, 1191, 83, 1102, 1588, 1596, 342, 1726, 970, 928, 1650, 1667, 341, 222, 750, 946, 477, 430, 1784, 977, 1601, 296, 1310, 214, 842, 1076, 650, 1690, 483, 1844, 1551, 1771, 751, 

/var/folders/s0/79hxvcq50xzchv821cc55yxw0000gp/T/ipykernel_96553/657226236.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_marked[f'satis_{model}'][i] = 1


[1171, 931, 512, 1706, 733, 678, 353, 1673, 1578, 904, 1071, 1466, 1053, 1428, 202, 114, 649, 160, 1590, 706, 537, 865, 623, 616, 1549, 995, 921, 1168, 589, 1086, 1283, 139, 289, 1695, 1491, 1288, 1042, 1336, 1030, 1598, 346, 1519, 561, 1380, 129, 1461, 1402, 627, 1566, 201, 440, 156, 538, 314, 1527, 282, 768, 149, 366, 460, 878, 1368, 1594, 420, 1271, 1415, 638, 744, 779, 35, 1172, 287, 630, 82, 1728, 1759, 1340, 1277, 599, 947, 421, 684, 643, 189, 1429, 1144, 1264, 1126, 77, 1212, 1473, 609, 1267, 1056, 1240, 1828, 1712, 1774, 1597, 788, 1221, 1583, 565, 238, 1459, 178, 1037, 1593, 988, 1304, 1255, 368, 471, 953]
[1746, 1345, 574, 338, 806, 805, 693, 513, 1584, 1205, 1586, 1045, 1096]
[1294, 1397, 1384, 219, 729, 1735, 44, 385, 752, 1847, 1420, 742, 1562, 70, 566, 1090, 1498, 1170, 1470, 475, 1779, 1396, 484, 1522, 1718, 683, 1501, 1219, 620, 1474, 700, 767, 809, 1047, 1185, 597, 1328, 557, 1683, 1012, 830, 1766, 662, 1304, 351, 40, 1130, 488, 1262, 1059, 725, 1362, 134, 614, 1675, 4

/var/folders/s0/79hxvcq50xzchv821cc55yxw0000gp/T/ipykernel_96553/657226236.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_marked[f'satis_{model}'][i] = 1


[783, 1020, 962, 659, 86, 850, 1165, 1714, 1226, 1840, 1748, 633, 748, 680, 1372, 130, 413, 1769, 176, 1676, 248, 1108, 1371, 1258, 1216, 71, 1237, 876, 114, 1511, 992, 34, 1220, 243, 1647, 1721, 350, 362, 975, 38, 90, 989, 1146, 1572, 1195, 549, 504, 103, 382, 1413, 1218, 1585, 152, 560, 1173, 443, 563, 1479, 1723, 110, 1066, 1035, 464, 1725, 1011, 1664, 1217, 1223, 1637]
[690, 733, 678, 953, 353, 1673, 1578, 904, 1071, 1466, 1053, 1428, 174, 202, 377, 114, 649, 160, 1590, 706, 537, 865, 623, 140, 616, 1549, 76, 995, 921, 1073, 1168, 589, 1086, 1283, 289, 1695, 1491, 1288, 1042, 1336, 1030, 346, 561, 1380, 129, 1461, 1402, 627, 1821, 1566, 201, 440, 156, 538, 314, 282, 768, 149, 366, 460, 878, 1368, 420, 1271, 1415, 638, 744, 779, 35, 1172, 287, 630, 931, 82, 1728, 1759, 1340, 1277, 599, 421, 684, 643, 1429, 239, 1144, 1264, 1126, 77, 1212, 1473, 609, 1267, 1056, 1240, 1828, 1712, 1597, 788, 1221, 1583, 565, 238, 1459, 178, 1037, 1593, 988, 1255, 368, 471, 1580, 183]
[1746, 1345, 574,

In [124]:
df_marked

,_unit_id,_golden,_unit_state,_trusted_judgments,confidence,confidence:stddev,relation,relation:confidence,orig__golden,context,...,gpt_davinci,gpt_35,gpt_35_0613,selected,sen1_inx,sen2_inx,satis_relation_AAF,satis_gpt_davinci,satis_gpt_35,satis_gpt_35_0613
0,3085729585,True,golden,65,4.45,0.702,neither,0.8807,True,"<strong>Kamala Harris:</strong> ""Please vote.""...",...,2,2,2,1,738,1547,0,0,0,0
1,3085729586,True,golden,70,4.46,0.750,neither,0.9203,True,"<strong>Donald Trump:</strong> ""Look what happ...",...,2,2,2,1,1384,352,0,0,0,0
2,3085729587,True,golden,60,3.93,0.772,neither,1.0000,True,"<strong>Donald Trump:</strong> ""And you’ll get...",...,0,0,0,1,143,1631,0,0,0,0
3,3085729588,False,golden,6,4.33,0.471,attack,0.4744,True,"<strong>Kristen Welker:</strong> ""I have one f...",...,0,0,0,1,1062,1801,0,0,0,0
4,3085729589,True,golden,67,4.57,0.628,support,0.9631,True,"<strong>Susan Page:</strong> ""So, tell us spec...",...,2,2,2,1,353,230,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1447,3101684951,False,finalized,6,4.17,0.687,neither,0.8339,NaN,"<strong>Donald Trump:</strong> ""He was thrown ...",...,0,0,0,1,86,1372,0,0,0,0
1448,3101684954,False,finalized,4,4.00,0.707,neither,0.7530,NaN,"<strong>Joe Biden:</strong> ""And what’s happen...",...,0,0,0,1,475,371,1,1,1,1
1449,3101684963,False,finalized,3,3.00,0.000,neither,1.0000,NaN,"<strong>Donald Trump:</strong> ""I don’t love w...",...,0,0,0,1,1003,569,1,1,1,1
1450,3101685003,False,finalized,3,3.00,0.000,neither,1.0000,NaN,"<strong>Donald Trump:</strong> ""I am under aud...",...,0,0,0,1,941,312,1,1,1,1


In [125]:
print("satis_relation_AAF", df_marked.value_counts("satis_relation_AAF"))
print("satis_gpt_davinci", df_marked.value_counts("satis_gpt_davinci"))
print("satis_gpt_35", df_marked.value_counts("satis_gpt_35"))
print("satis_gpt_35_0613", df_marked.value_counts("satis_gpt_35_0613"))

satis_relation_AAF satis_relation_AAF
0    1360
1      92
dtype: int64
satis_gpt_davinci satis_gpt_davinci
0    1364
1      88
dtype: int64
satis_gpt_35 satis_gpt_35
0    1367
1      85
dtype: int64
satis_gpt_35_0613 satis_gpt_35_0613
0    1368
1      84
dtype: int64


In [127]:
df_marked.value_counts("sen1_inx")

sen1_inx
475     10
114     10
1346     6
1493     5
395      5
        ..
682      1
683      1
685      1
687      1
1849     1
Length: 1141, dtype: int64

In [126]:
df_marked.to_csv('Web-Dash\mvp2data.csv', index=False)